In [2]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet121', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet169', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet201', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'densenet161', pretrained=True)
model.eval()

Using cache found in C:\Users\fishd/.cache\torch\hub\pytorch_vision_v0.6.0


enseLayer(
        (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(288, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer3): _DenseLayer(
        (norm1)

In [3]:
import glob
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

groups_folder_path = './Dataset/Train/'
categories = ["male","female"]
num_classes = len(categories)

image_w = 28
image_h = 28
  
X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            # print(image_dir+filename)
            img = cv2.imread(image_dir+filename)
            img = cv2.resize(img, None, fx=image_w/img.shape[0], fy=image_h/img.shape[1])
            X.append(img/256)
            Y.append(label)
 
X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)



In [4]:
X_train[1]

array([[[0.1484375 , 0.12109375, 0.03125   ],
        [0.109375  , 0.08203125, 0.        ],
        [0.10546875, 0.09765625, 0.00390625],
        ...,
        [0.5625    , 0.49609375, 0.4453125 ],
        [0.1328125 , 0.04296875, 0.01171875],
        [0.37109375, 0.25390625, 0.1875    ]],

       [[0.17578125, 0.125     , 0.0234375 ],
        [0.1484375 , 0.09765625, 0.        ],
        [0.16015625, 0.09375   , 0.        ],
        ...,
        [0.54296875, 0.48046875, 0.42578125],
        [0.234375  , 0.140625  , 0.11328125],
        [0.37109375, 0.25390625, 0.1875    ]],

       [[0.16796875, 0.12109375, 0.015625  ],
        [0.1796875 , 0.125     , 0.01953125],
        [0.2109375 , 0.11328125, 0.01171875],
        ...,
        [0.52734375, 0.4609375 , 0.4140625 ],
        [0.30859375, 0.20703125, 0.1640625 ],
        [0.37109375, 0.265625  , 0.19140625]],

       ...,

       [[0.90625   , 0.96484375, 0.9921875 ],
        [0.91796875, 0.9765625 , 0.99609375],
        [0.9140625 , 0

In [7]:

model = model((32,32,3), 10, 'DenseNet-201')

optimizer = SGD(lr=0.1, decay=1e-3, momentum=0.9, nesterov=True)

callbacks_list = [LearningRateSchedule([30, 60])]

model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])


history = model.fit(x_train, y_train, batch_size=256, epochs=90, validation_split=0.2, callbacks=callbacks_list)

TypeError: forward() takes 2 positional arguments but 4 were given

In [17]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [21]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 6.00 GiB total capacity; 0 bytes already allocated; 0 bytes free; 0 bytes reserved in total by PyTorch)